In [ ]:
from datasets import load_dataset
ds = load_dataset("Synanthropic/reading-analog-gauge")

## Unoptimized for mac m1 GPU

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from math import atan2, degrees
import pickle
from datasets import load_dataset
import matplotlib.pyplot as plt

class GaugeReader:
    def __init__(self):
        self.input_shape = (224, 224, 3)
        self.min_value = 0
        self.max_value = 10
        
    def build_model(self):
        # Modified model architecture to predict single value
        model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=self.input_shape),
            tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(1)  # Single output for gauge reading
        ])
        
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='mse',
            metrics=['mae']
        )
        
        return model

    def prepare_dataset(self):
        # Load dataset from HuggingFace
        print("Loading dataset...")
        dataset = load_dataset("Synanthropic/reading-analog-gauge")
        train_ds = dataset['train']
        
        def preprocess_single_example(example):
            # Convert image to numpy array
            image = np.array(example['image'])
            
            # Resize image
            image = cv2.resize(image, (self.input_shape[0], self.input_shape[1]))
            
            # Normalize image
            image = image.astype(np.float32) / 255.0
            
            # Get label
            label = np.array(example['label'], dtype=np.float32)
            
            return {
                'image': image,
                'label': label
            }
        
        print("Processing examples...")
        # Process all examples
        processed_data = [preprocess_single_example(example) for example in train_ds]
        
        # Separate images and labels
        images = np.array([example['image'] for example in processed_data])
        labels = np.array([example['label'] for example in processed_data])
        
        print(f"Dataset shape - Images: {images.shape}, Labels: {labels.shape}")
        
        # Create TensorFlow dataset
        dataset = tf.data.Dataset.from_tensor_slices((images, labels))
        
        # Batch and shuffle
        dataset = dataset.shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
        
        return dataset

    def train_model(self, epochs=10):
        # Build model
        model = self.build_model()
        
        print("Preparing dataset...")
        train_ds = self.prepare_dataset()
        print("Dataset prepared successfully!")
        
        # Train model
        print("Starting training...")
        history = model.fit(
            train_ds,
            epochs=epochs,
            callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='loss',
                    patience=3,
                    restore_best_weights=True
                ),
                tf.keras.callbacks.ReduceLROnPlateau(
                    monitor='loss',
                    factor=0.5,
                    patience=2
                )
            ]
        )
        
        return model, history

    def convert_to_tflite(self, model):
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        tflite_model = converter.convert()
        return tflite_model

    def save_model(self, tflite_model, model_path):
        with open(model_path, 'wb') as f:
            pickle.dump(tflite_model, f)
    
    def predict_value(self, model, image_path):
        # Load and preprocess image
        image = cv2.imread(image_path)
        image = cv2.resize(image, (self.input_shape[0], self.input_shape[1]))
        image = image.astype(np.float32) / 255.0
        
        # Add batch dimension
        image = np.expand_dims(image, 0)
        
        # Predict
        prediction = model.predict(image)
        return float(prediction[0][0])

    def visualize_prediction(self, image_path, prediction):
        # Load and resize image
        image = cv2.imread(image_path)
        image = cv2.resize(image, (self.input_shape[0], self.input_shape[1]))
        
        plt.figure(figsize=(10, 10))
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.title(f'Predicted Value: {prediction:.2f}')
        plt.axis('off')
        plt.show()

def main():
    # Initialize GaugeReader
    gauge_reader = GaugeReader()
    
    # Train model
    print("Training model...")
    model, history = gauge_reader.train_model(epochs=10)
    
    # Convert to TFLite
    print("Converting to TFLite...")
    tflite_model = gauge_reader.convert_to_tflite(model)
    
    # Save model
    print("Saving model...")
    gauge_reader.save_model(tflite_model, 'gauge_reader_model.tflite')
    
    # Plot training history
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['mae'], label='MAE')
    plt.title('Model MAE')
    plt.xlabel('Epoch')
    plt.ylabel('MAE')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()

In [ ]:
# Check dataset structure
dataset = load_dataset("Synanthropic/reading-analog-gauge")
train_ds = dataset['train']
example = train_ds[0]
print("Dataset keys:", example.keys())

## Optimized for mac m1 GPU

In [2]:
import tensorflow as tf
import numpy as np
import cv2
from math import atan2, degrees
import pickle
from datasets import load_dataset
import matplotlib.pyplot as plt
import gc  # Garbage collector
import os

class GaugeReader:
    def __init__(self):
        self.input_shape = (160, 160, 3)
        self.min_value = 0
        self.max_value = 10
        self.batch_size = 16
        self.checkpoint_dir = 'training_checkpoints'
        os.makedirs(self.checkpoint_dir, exist_ok=True)
        
    def build_model(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=self.input_shape),
            tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(1)
        ])
        
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='mse',
            metrics=['mae']
        )
        
        return model

    def prepare_dataset(self):
        print("Loading dataset...")
        dataset = load_dataset(
            "Synanthropic/reading-analog-gauge",
            split='train'
        )
        
        print(f"Total examples in dataset: {len(dataset)}")
        
        def preprocess_example(example):
            # Load and resize image
            image = np.array(example['image'])
            image = cv2.resize(image, (self.input_shape[0], self.input_shape[1]))
            image = image.astype(np.float32) / 255.0
            label = float(example['label'])
            return image, label

        def generator():
            for example in dataset:
                image, label = preprocess_example(example)
                yield image, label

        tf_dataset = tf.data.Dataset.from_generator(
            generator,
            output_signature=(
                tf.TensorSpec(shape=self.input_shape, dtype=tf.float32),
                tf.TensorSpec(shape=(), dtype=tf.float32)
            )
        )

        tf_dataset = tf_dataset.shuffle(1000)
        tf_dataset = tf_dataset.batch(self.batch_size)
        tf_dataset = tf_dataset.prefetch(tf.data.AUTOTUNE)

        steps_per_epoch = len(dataset) // self.batch_size

        return tf_dataset, steps_per_epoch

    def train_model(self, epochs=10):
        # Build model
        model = self.build_model()
        
        print("Preparing dataset...")
        train_ds, steps_per_epoch = self.prepare_dataset()
        print(f"Dataset prepared successfully! Steps per epoch: {steps_per_epoch}")
        
        # Define checkpoint paths with .keras extension
        checkpoint_path = os.path.join(self.checkpoint_dir, "epoch_{epoch:02d}.keras")
        best_model_path = os.path.join(self.checkpoint_dir, "best_model.keras")
        
        # Train model
        print("Starting training...")
        history = model.fit(
            train_ds,
            epochs=epochs,
            steps_per_epoch=steps_per_epoch,
            callbacks=[
                # Save after every epoch
                tf.keras.callbacks.ModelCheckpoint(
                    filepath=checkpoint_path,
                    save_weights_only=False,
                    save_freq='epoch'
                ),
                # Save best model based on loss
                tf.keras.callbacks.ModelCheckpoint(
                    filepath=best_model_path,
                    save_best_only=True,
                    monitor='loss',
                    mode='min',
                    save_weights_only=False
                ),
                tf.keras.callbacks.EarlyStopping(
                    monitor='loss',
                    patience=3,
                    restore_best_weights=True
                ),
                tf.keras.callbacks.ReduceLROnPlateau(
                    monitor='loss',
                    factor=0.5,
                    patience=2
                ),
                # Memory cleanup callback
                tf.keras.callbacks.LambdaCallback(
                    on_epoch_end=lambda epoch, logs: gc.collect()
                )
            ]
        )
        
        return model, history

    def load_latest_checkpoint(self):
        """Load the latest checkpoint if it exists."""
        checkpoints = [d for d in os.listdir(self.checkpoint_dir) 
                      if d.startswith('epoch_') and d.endswith('.keras')]
        if not checkpoints:
            return None
            
        latest_checkpoint = max(checkpoints)
        checkpoint_path = os.path.join(self.checkpoint_dir, latest_checkpoint)
        print(f"Loading checkpoint: {checkpoint_path}")
        return tf.keras.models.load_model(checkpoint_path)

    def convert_to_tflite(self, model):
        print("Converting to TFLite format...")
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        tflite_model = converter.convert()
        return tflite_model

    def save_model(self, tflite_model, model_path):
        with open(model_path, 'wb') as f:
            pickle.dump(tflite_model, f)

def main():
    # Initialize GaugeReader
    gauge_reader = GaugeReader()
    
    try:
        # Check for existing checkpoints
        existing_model = gauge_reader.load_latest_checkpoint()
        if existing_model:
            print("Resuming from previous checkpoint...")
            model = existing_model
        else:
            # Train model
            print("Training new model...")
            model, history = gauge_reader.train_model(epochs=10)
        
        # Convert to TFLite
        print("Converting to TFLite...")
        tflite_model = gauge_reader.convert_to_tflite(model)
        
        # Save model
        print("Saving model...")
        gauge_reader.save_model(tflite_model, 'gauge_reader_model.tflite')
        
        # Plot training history if available
        if 'history' in locals():
            plt.figure(figsize=(10, 5))
            plt.subplot(1, 2, 1)
            plt.plot(history.history['loss'], label='Loss')
            plt.title('Model Loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.legend()
            
            plt.subplot(1, 2, 2)
            plt.plot(history.history['mae'], label='MAE')
            plt.title('Model MAE')
            plt.xlabel('Epoch')
            plt.ylabel('MAE')
            plt.legend()
            
            plt.tight_layout()
            plt.show()
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        # Clean up memory in case of error
        gc.collect()

if __name__ == "__main__":
    main()

Loading checkpoint: training_checkpoints/epoch_04.keras


2024-12-30 18:28:27.455764: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-12-30 18:28:27.455783: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-12-30 18:28:27.455786: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-12-30 18:28:27.455821: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-30 18:28:27.455835: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Resuming from previous checkpoint...
Converting to TFLite...
Converting to TFLite format...
INFO:tensorflow:Assets written to: /var/folders/rt/0x35vmdd32ldd69_8j6wrvgc0000gn/T/tmp1s0ea92r/assets


INFO:tensorflow:Assets written to: /var/folders/rt/0x35vmdd32ldd69_8j6wrvgc0000gn/T/tmp1s0ea92r/assets


: 

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15670774299620544029
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
locality {
  bus_id: 1
}
incarnation: 13422472795634595793
physical_device_desc: "device: 0, name: METAL, pci bus id: <undefined>"
xla_global_id: -1
]


2024-12-30 18:30:26.618328: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-12-30 18:30:26.618445: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-12-30 18:30:26.618497: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-12-30 18:30:26.618562: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-30 18:30:26.618591: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## optimized for RTX 3060 gpu

In [2]:
import tensorflow as tf
import numpy as np
import cv2
from math import atan2, degrees
import pickle
from datasets import load_dataset
import matplotlib.pyplot as plt
from tensorflow.keras import mixed_precision

class GaugeReader:
    def __init__(self):
        self.input_shape = (160, 160, 3)  # Reduced input size
        self.min_value = 0
        self.max_value = 10
        self.batch_size = 32  # Smaller batch size to reduce memory

    def build_model(self):
        # Lighter model architecture with fewer parameters
        model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=self.input_shape),
            
            # First conv block - reduced filters
            tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D(),
            
            # Second conv block
            tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D(),
            
            # Third conv block
            tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D(),
            
            # Fourth conv block
            tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.GlobalAveragePooling2D(),
            
            # Reduced dense layers
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(1)
        ])
        
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        
        model.compile(
            optimizer=optimizer,
            loss='mse',
            metrics=['mae']
        )
        
        return model

    def prepare_dataset(self):
        print("Loading dataset...")
        # Load dataset in streaming mode
        dataset = load_dataset(
            "Synanthropic/reading-analog-gauge",
            streaming=True
        )
        train_ds = dataset['train']
        
        def preprocess_example(example):
            image = np.array(example['image'])
            image = cv2.resize(image, (self.input_shape[0], self.input_shape[1]))
            image = (image.astype(np.float32) - 127.5) / 127.5  # Normalize to [-1, 1]
            return image, example['label']
        
        # Create streaming datasets with generators
        def generate_examples(split):
            for example in split:
                yield preprocess_example(example)
        
        # Calculate approximate sizes
        total_size = 10000  # Approximate dataset size
        val_size = int(0.1 * total_size)
        
        # Create datasets using from_generator
        train_dataset = tf.data.Dataset.from_generator(
            lambda: generate_examples(train_ds.take(total_size - val_size)),
            output_signature=(
                tf.TensorSpec(shape=self.input_shape, dtype=tf.float32),
                tf.TensorSpec(shape=(), dtype=tf.float32)
            )
        )
        
        val_dataset = tf.data.Dataset.from_generator(
            lambda: generate_examples(train_ds.skip(total_size - val_size).take(val_size)),
            output_signature=(
                tf.TensorSpec(shape=self.input_shape, dtype=tf.float32),
                tf.TensorSpec(shape=(), dtype=tf.float32)
            )
        )
        
        # Optimize for performance while maintaining memory efficiency
        train_dataset = (train_dataset
            .shuffle(1000)  # Reduced buffer size
            .batch(self.batch_size)
            .prefetch(tf.data.AUTOTUNE)
        )
        
        val_dataset = (val_dataset
            .batch(self.batch_size)
            .prefetch(tf.data.AUTOTUNE)
        )
        
        return train_dataset, val_dataset

    def train_model(self, epochs=20):
        # Configure memory growth
        for device in tf.config.list_physical_devices('GPU'):
            tf.config.experimental.set_memory_growth(device, True)
        
        model = self.build_model()
        train_ds, val_ds = self.prepare_dataset()
        
        # Add TensorBoard callback for memory monitoring
        tensorboard_callback = tf.keras.callbacks.TensorBoard(
            log_dir='./logs',
            profile_batch='500,520'  # Profile a few batches
        )
        
        history = model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=epochs,
            callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=5,
                    restore_best_weights=True
                ),
                tf.keras.callbacks.ReduceLROnPlateau(
                    monitor='val_loss',
                    factor=0.5,
                    patience=3,
                    min_lr=1e-6
                ),
                tensorboard_callback
            ]
        )
        
        return model, history

    def convert_to_tflite(self, model):
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.target_spec.supported_types = [tf.float16]  # Use FP16 quantization
        tflite_model = converter.convert()
        return tflite_model

    def save_model(self, tflite_model, model_path):
        with open(model_path, 'wb') as f:
            f.write(tflite_model)  # Save directly without pickle

def main():
    gauge_reader = GaugeReader()
    model, history = gauge_reader.train_model(epochs=20)
    tflite_model = gauge_reader.convert_to_tflite(model)
    gauge_reader.save_model(tflite_model, 'gauge_reader_model.tflite')
    
    # Plot with reduced memory usage
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train')
    plt.plot(history.history['val_loss'], label='Val')
    plt.title('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['mae'], label='Train')
    plt.plot(history.history['val_mae'], label='Val')
    plt.title('MAE')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()

if __name__ == "__main__":
    main()

/home/faizal/work/gnprc-domi-app/tf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
I0000 00:00:1735302132.971772    6181 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9812 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:2b:00.0, compute capability: 8.6


Loading dataset...


Repo card metadata block was not found. Setting CardData to empty.
2024-12-27 17:52:28.489984: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:103] Profiler session initializing.
2024-12-27 17:52:28.490012: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:118] Profiler session started.
2024-12-27 17:52:28.490036: I external/local_xla/xla/backends/profiler/gpu/cupti_tracer.cc:1006] Profiler found 1 GPUs
2024-12-27 17:52:28.518519: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:130] Profiler session tear down.
2024-12-27 17:52:28.518593: I external/local_xla/xla/backends/profiler/gpu/cupti_tracer.cc:1213] CUPTI activity buffer flushed


Epoch 1/20


2024-12-27 17:52:40.442151: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:4: Filling up shuffle buffer (this may take a while): 5 of 1000
2024-12-27 17:52:50.781367: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:4: Filling up shuffle buffer (this may take a while): 12 of 1000
2024-12-27 17:53:10.919037: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:4: Filling up shuffle buffer (this may take a while): 26 of 1000
2024-12-27 17:53:21.097728: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:4: Filling up shuffle buffer (this may take a while): 33 of 1000
2024-12-27 17:53:40.070189: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:4: Filling up shuffle buffer (this may take a while): 46 of 1000
2024-12-27 17:53:50.271973: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:4: Filling up shuffle buffer (this may take a while): 53 o

      1/Unknown 1539s 1539s/step - loss: 0.0012 - mae: 0.0280

I0000 00:00:1735303687.475291    7506 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


     63/Unknown 4542s 48s/step - loss: 0.0024 - mae: 0.0211

KeyboardInterrupt: 

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2024-12-27 17:51:51.975267: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735302112.075544    6181 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735302112.103413    6181 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-27 17:51:52.359926: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Num GPUs Available:  1


In [ ]:
import tensorflow as tf
import numpy as np
from datasets import load_dataset
import matplotlib.pyplot as plt
from tensorflow.keras import mixed_precision

class GaugeReader:
    def __init__(self):
        self.input_shape = (160, 160, 3)
        self.min_value = 0
        self.max_value = 10
        self.batch_size = 64  # Increased but still conservative for 12GB VRAM
        
        # Enable mixed precision for better memory efficiency
        mixed_precision.set_global_policy('mixed_float16')

    def build_model(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=self.input_shape),
            
            tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(),
            
            tf.keras.layers.GlobalAveragePooling2D(),
            
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(1)
        ])
        
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        
        model.compile(
            optimizer=optimizer,
            loss='mse',
            metrics=['mae']
        )
        
        return model

    def prepare_dataset(self):
        print("Loading dataset...")
        dataset = load_dataset(
            "Synanthropic/reading-analog-gauge",
            streaming=True
        )
        train_ds = dataset['train']
        
        @tf.function
        def preprocess_image(image):
            image = tf.image.resize(image, [self.input_shape[0], self.input_shape[1]])
            image = tf.cast(image, tf.float32) / 127.5 - 1
            return image

        def generate_examples(split):
            for example in split:
                # Convert PIL image to numpy array
                image = np.array(example['image'])
                # Convert to tensor and preprocess
                image = preprocess_image(image)
                yield image, example['label']

        # Calculate sizes
        total_size = 10000
        val_size = int(0.1 * total_size)
        
        # Create datasets
        train_dataset = tf.data.Dataset.from_generator(
            lambda: generate_examples(train_ds.take(total_size - val_size)),
            output_signature=(
                tf.TensorSpec(shape=self.input_shape, dtype=tf.float32),
                tf.TensorSpec(shape=(), dtype=tf.float32)
            )
        )
        
        val_dataset = tf.data.Dataset.from_generator(
            lambda: generate_examples(train_ds.skip(total_size - val_size).take(val_size)),
            output_signature=(
                tf.TensorSpec(shape=self.input_shape, dtype=tf.float32),
                tf.TensorSpec(shape=(), dtype=tf.float32)
            )
        )
        
        # Optimize the input pipeline
        train_dataset = (train_dataset
            .cache()  # Cache after preprocessing
            .shuffle(1000)
            .batch(self.batch_size)
            .prefetch(tf.data.AUTOTUNE)
        )
        
        val_dataset = (val_dataset
            .batch(self.batch_size)
            .prefetch(tf.data.AUTOTUNE)
        )
        
        return train_dataset, val_dataset

    def train_model(self, epochs=20):
        # Configure GPU memory growth
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
        
        model = self.build_model()
        train_ds, val_ds = self.prepare_dataset()
        
        tensorboard_callback = tf.keras.callbacks.TensorBoard(
            log_dir='./logs',
            profile_batch='100,120'
        )
        
        history = model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=epochs,
            callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=5,
                    restore_best_weights=True
                ),
                tf.keras.callbacks.ReduceLROnPlateau(
                    monitor='val_loss',
                    factor=0.5,
                    patience=3,
                    min_lr=1e-6
                ),
                tensorboard_callback
            ]
        )
        
        return model, history

    def convert_to_tflite(self, model):
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.target_spec.supported_types = [tf.float16]
        tflite_model = converter.convert()
        return tflite_model

    def save_model(self, tflite_model, model_path):
        with open(model_path, 'wb') as f:
            f.write(tflite_model)

def main():
    gauge_reader = GaugeReader()
    model, history = gauge_reader.train_model(epochs=20)
    tflite_model = gauge_reader.convert_to_tflite(model)
    gauge_reader.save_model(tflite_model, 'gauge_reader_model.tflite')
    
    # Plot with reduced memory usage
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train')
    plt.plot(history.history['val_loss'], label='Val')
    plt.title('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['mae'], label='Train')
    plt.plot(history.history['val_mae'], label='Val')
    plt.title('MAE')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()

if __name__ == "__main__":
    main()

Loading dataset...


Repo card metadata block was not found. Setting CardData to empty.


Epoch 1/20


2024-12-27 19:12:05.432159: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:103] Profiler session initializing.
2024-12-27 19:12:05.432183: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:118] Profiler session started.
2024-12-27 19:12:05.437055: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:130] Profiler session tear down.
2024-12-27 19:12:05.440574: I external/local_xla/xla/backends/profiler/gpu/cupti_tracer.cc:1213] CUPTI activity buffer flushed
2024-12-27 19:12:17.525228: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:19: Filling up shuffle buffer (this may take a while): 4 of 1000
2024-12-27 19:12:27.841763: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:19: Filling up shuffle buffer (this may take a while): 11 of 1000
2024-12-27 19:12:48.472231: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:19: Filling up shuffle buffer (this may take a while): 24 of 1000
2024-12-2

     11/Unknown 2565s 97s/step - loss: 2.6546 - mae: 1.1359

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 01e050d6-887c-4282-a4c1-2fb05ea4c2a4)')' thrown while requesting GET https://huggingface.co/datasets/Synanthropic/reading-analog-gauge/resolve/f9b31b44c9a693eddcc9c93a247ad30c3005ea07/corners.zip
Retrying in 1s [Retry 1/5].
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /datasets/Synanthropic/reading-analog-gauge/resolve/f9b31b44c9a693eddcc9c93a247ad30c3005ea07/corners.zip (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7788333d4200>: Failed to resolve \'huggingface.co\' ([Errno -3] Temporary failure in name resolution)"))'), '(Request ID: 1a075cb6-9d14-46ac-9005-ab8914b3560c)')' thrown while requesting GET https://huggingface.co/datasets/Synanthropic/reading-analog-gauge/resolve/f9b31b44c9a693eddcc9c93a247ad30c3005ea07/corners.zip
Retrying in 2s [Retry 2/5].
'(MaxRetryError(

     31/Unknown 4583s 100s/step - loss: 1.6418 - mae: 0.8470

## Working on roboflow dataset

In [15]:
import torch
from ultralytics import YOLO
import cv2
import numpy as np
import math
from pathlib import Path
import yaml
import os

class GaugeReaderYOLO:
    def __init__(self):
        self.model = None
        
    def update_yaml_paths(self, data_yaml_path):
        """Update data.yaml with correct absolute paths"""
        # Get the dataset root directory
        dataset_dir = Path(data_yaml_path).parent
        
        # Read existing yaml
        with open(data_yaml_path, 'r') as f:
            data_yaml = yaml.safe_load(f)
        
        # Update paths with absolute paths
        data_yaml['path'] = str(dataset_dir.absolute())
        data_yaml['train'] = str((dataset_dir / 'train' / 'images').absolute())
        data_yaml['val'] = str((dataset_dir / 'valid' / 'images').absolute())
        
        # Verify paths exist
        for path_key in ['train', 'val']:
            path = Path(data_yaml[path_key])
            if not path.exists():
                raise FileNotFoundError(f"Path {path} does not exist!")
        
        # Save updated yaml
        updated_yaml_path = dataset_dir / 'updated_data.yaml'
        with open(updated_yaml_path, 'w') as f:
            yaml.dump(data_yaml, f)
            
        return str(updated_yaml_path)
    
    def train(self, data_yaml_path, epochs=50):
        """Train YOLOv8 model"""
        # Update yaml paths
        print("Updating dataset paths...")
        updated_yaml_path = self.update_yaml_paths(data_yaml_path)
        
        print(f"Training with dataset config: {updated_yaml_path}")
        
        # Initialize model
        self.model = YOLO('yolov8n.pt')  # use YOLOv8 nano model
        
        # Train the model
        try:
            self.model.train(
                data=updated_yaml_path,
                epochs=epochs,
                imgsz=640,
                batch=16,
                device=0,  # Use GPU
                patience=10,  # Early stopping
                save=True,
                project='gauge_detection'
            )
        except Exception as e:
            print(f"Error during training: {str(e)}")
            # Print the content of the yaml file for debugging
            with open(updated_yaml_path, 'r') as f:
                print("\nDataset YAML contents:")
                print(f.read())
            raise
            
    def calculate_angle(self, center, needle_tip):
        """Calculate angle between vertical line and needle"""
        dx = needle_tip[0] - center[0]
        dy = needle_tip[1] - center[1]
        angle = math.degrees(math.atan2(dy, dx))
        if angle < 0:
            angle += 360
        return angle

    def get_reading_from_angle(self, angle, min_angle=45, max_angle=315, min_value=0, max_value=100):
        """Convert angle to gauge reading"""
        # Normalize the angle to the gauge's range
        if angle > max_angle:
            angle -= 360
            
        # Calculate reading using linear interpolation
        angle_range = max_angle - min_angle
        value_range = max_value - min_value
        
        reading = ((angle - min_angle) / angle_range) * value_range + min_value
        return round(reading, 1)

    def predict(self, image_path):
        """Predict gauge reading from image"""
        if self.model is None:
            raise ValueError("Model not trained. Please train the model first.")
            
        # Read image
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Could not read image at {image_path}")
            
        # Get predictions
        results = self.model(image)
        
        # Initialize variables
        center = None
        needle_tip = None
        gauge_box = None
        
        # Process predictions
        for result in results:
            boxes = result.boxes
            for box in boxes:
                # Get class and confidence
                cls = int(box.cls)
                conf = float(box.conf)
                
                if conf > 0.4:  # Confidence threshold
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    
                    if cls == 0:  # Center
                        center = (int((x1 + x2) / 2), int((y1 + y2) / 2))
                    elif cls == 1:  # Gauge
                        gauge_box = (x1, y1, x2, y2)
                    elif cls == 2:  # Needle
                        needle_tip = (int((x1 + x2) / 2), int((y1 + y2) / 2))
        
        if center and needle_tip:
            angle = self.calculate_angle(center, needle_tip)
            reading = self.get_reading_from_angle(angle)
            
            return {
                'center': center,
                'needle_tip': needle_tip,
                'gauge_box': gauge_box,
                'angle': angle,
                'reading': reading,
                'image': image
            }
        return None

    def visualize_results(self, results):
        """Visualize detection results"""
        if results is None:
            return None
            
        output = results['image'].copy()
        
        if results['gauge_box']:
            x1, y1, x2, y2 = results['gauge_box']
            cv2.rectangle(output, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        if results['center']:
            cv2.circle(output, results['center'], 5, (0, 0, 255), -1)
            
        if results['needle_tip']:
            cv2.circle(output, results['needle_tip'], 5, (255, 0, 0), -1)
            cv2.line(output, results['center'], results['needle_tip'], (255, 0, 0), 2)
            
        # Add text with reading
        cv2.putText(output, 
                   f"Reading: {results['reading']:.1f}", 
                   (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 
                   1, 
                   (0, 0, 255), 
                   2)
        
        return output

In [16]:
# Initialize the gauge reader
gauge_reader = GaugeReaderYOLO()

# Train the model using your existing YOLO format dataset
data_yaml_path = "/home/faizal/work/gnprc-domi-app/datasets/Analog_Gauge_Meter.v13-v06-crop.yolov8/data.yaml"
gauge_reader.train(data_yaml_path, epochs=50)

# Make predictions
results = gauge_reader.predict('/home/faizal/work/gnprc-domi-app/Model/guage_image.jpg')
if results:
    # Visualize results
    output_image = gauge_reader.visualize_results(results)
    cv2.imwrite('result.jpg', output_image)
    print(f"Gauge Reading: {results['reading']}")

Updating dataset paths...
Training with dataset config: /home/faizal/work/gnprc-domi-app/datasets/Analog_Gauge_Meter.v13-v06-crop.yolov8/updated_data.yaml
Ultralytics 8.3.55 🚀 Python-3.12.3 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3060, 12030MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/faizal/work/gnprc-domi-app/datasets/Analog_Gauge_Meter.v13-v06-crop.yolov8/updated_data.yaml, epochs=50, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=gauge_detection, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_strid

train: Scanning /home/faizal/work/gnprc-domi-app/datasets/Analog_Gauge_Meter.v13-v06-crop.yolov8/train/labels.cache... 390 images, 0 backgrounds, 0 corrupt: 100%|██████████| 390/390 [00:00<?, ?it/s]
val: Scanning /home/faizal/work/gnprc-domi-app/datasets/Analog_Gauge_Meter.v13-v06-crop.yolov8/valid/labels.cache... 33 images, 0 backgrounds, 0 corrupt: 100%|██████████| 33/33 [00:00<?, ?it/s]


Plotting labels to gauge_detection/train7/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to gauge_detection/train7
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      2.32G     0.8365      2.367      1.124         25        640: 100%|██████████| 25/25 [00:03<00:00,  8.13it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00, 13.62it/s]

                   all         33         99          1      0.322      0.332      0.308


...

      50/50      2.27G     0.3225     0.2431     0.8035         18        640: 100%|██████████| 25/25 [00:02<00:00,  9.53it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00, 15.29it/s]

                   all         33         99      0.996          1      0.995      0.884



50 epochs completed in 0.076 hours.
Optimizer stripped from gauge_detection/train7/weights/last.pt, 6.2MB
Optimizer stripped from gauge_detection/train7/weights/best.pt, 6.2MB

Validating gauge_detection/train7/weights/best.pt...
Ultralytics 8.3.55 🚀 Python-3.12.3 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3060, 12030MiB)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00, 13.14it/s]


                   all         33         99      0.996          1      0.995      0.885
                Center         33         33          1          1      0.995      0.796
                 Gauge         33         33       0.99          1      0.995      0.992
                Needle         33         33      0.998          1      0.995      0.868
Speed: 0.2ms preprocess, 1.8ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to gauge_detection/train7

0: 640x640 2 Centers, 1 Gauge, 1 Needle, 3.6ms
Speed: 1.3ms preprocess, 3.6ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)
Gauge Reading: 45.7


In [1]:
from ultralytics import YOLO
import cv2
import math
import os
import numpy as np

class GaugeReader:
    def __init__(self, model_path):
        try:
            self.model = YOLO(model_path)
            print(f"Model loaded successfully from: {model_path}")
        except Exception as e:
            raise Exception(f"Failed to load model: {str(e)}")
        
    def calculate_angle(self, center, needle_tip):
        """Calculate angle between vertical line and needle"""
        dx = needle_tip[0] - center[0]
        dy = needle_tip[1] - center[1]
        angle = math.degrees(math.atan2(dy, dx))
        if angle < 0:
            angle += 360
        return angle

    def get_reading_from_angle(self, angle, min_angle=45, max_angle=315, min_value=0, max_value=100):
        """Convert angle to gauge reading"""
        # Normalize the angle to the gauge's range
        if angle > max_angle:
            angle -= 360
            
        # Calculate reading using linear interpolation
        angle_range = max_angle - min_angle
        value_range = max_value - min_value
        
        reading = ((angle - min_angle) / angle_range) * value_range + min_value
        return round(reading, 1)

    def predict(self, image_path):
        """Predict gauge reading from image with debug information"""
        if not os.path.exists(image_path):
            raise FileNotFoundError(f"Image not found at: {image_path}")
            
        # Read and debug original image
        original_image = cv2.imread(image_path)
        if original_image is None:
            raise ValueError(f"Could not read image at {image_path}")
        
        # Save original image for debugging
        cv2.imwrite('debug_original.jpg', original_image)
        print(f"Original image shape: {original_image.shape}")
        
        # Get predictions with verbose mode
        results = self.model(original_image, verbose=True)
        
        # Print confidence scores for all detections
        print("\nAll detections (including low confidence):")
        for result in results:
            boxes = result.boxes
            for i, box in enumerate(boxes):
                cls = int(box.cls)
                conf = float(box.conf)
                class_name = ['Center', 'Gauge', 'Needle'][cls]
                print(f"Detection {i+1}: {class_name} - Confidence: {conf:.3f}")
        
        # Initialize variables
        center = None
        needle_tip = None
        gauge_box = None
        
        # Process predictions with lower confidence threshold for debugging
        for result in results:
            boxes = result.boxes
            for box in boxes:
                cls = int(box.cls)
                conf = float(box.conf)
                
                # Lower confidence threshold for debugging
                if conf > 0.3:  # Reduced from 0.5 for debugging
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    
                    if cls == 0:  # Center
                        center = (int((x1 + x2) / 2), int((y1 + y2) / 2))
                    elif cls == 1:  # Gauge
                        gauge_box = (x1, y1, x2, y2)
                    elif cls == 2:  # Needle
                        needle_tip = (int((x1 + x2) / 2), int((y1 + y2) / 2))
        
        # Create debug visualization
        debug_image = original_image.copy()
        
        # Draw all detections with confidence > 0.3
        for result in results:
            boxes = result.boxes
            for box in boxes:
                conf = float(box.conf)
                if conf > 0.3:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    cls = int(box.cls)
                    class_name = ['Center', 'Gauge', 'Needle'][cls]
                    color = [(0, 0, 255), (0, 255, 0), (255, 0, 0)][cls]
                    
                    cv2.rectangle(debug_image, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(debug_image, 
                              f"{class_name}: {conf:.2f}", 
                              (x1, y1-10), 
                              cv2.FONT_HERSHEY_SIMPLEX, 
                              0.5, 
                              color, 
                              2)
        
        # Save debug visualization
        cv2.imwrite('debug_detections.jpg', debug_image)
        
        if center and needle_tip:
            angle = self.calculate_angle(center, needle_tip)
            reading = self.get_reading_from_angle(angle)
            
            return {
                'center': center,
                'needle_tip': needle_tip,
                'gauge_box': gauge_box,
                'angle': angle,
                'reading': reading,
                'image': original_image
            }
        return None
    
    def visualize_results(self, results):
        """Visualize detection results"""
        if results is None:
            return None
            
        output = results['image'].copy()
        
        if results['gauge_box']:
            x1, y1, x2, y2 = results['gauge_box']
            cv2.rectangle(output, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        if results['center']:
            cv2.circle(output, results['center'], 5, (0, 0, 255), -1)
            
        if results['needle_tip']:
            cv2.circle(output, results['needle_tip'], 5, (255, 0, 0), -1)
            cv2.line(output, results['center'], results['needle_tip'], (255, 0, 0), 2)
            
        # Add text with reading
        cv2.putText(output, 
                f"Reading: {results['reading']:.1f}", 
                (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                1, 
                (0, 0, 255), 
                2)
        
        return output

# Run inference with debug output
if __name__ == "__main__":
    try:
        # Paths
        model_path = "/home/faizal/work/gnprc-domi-app/Model/gauge_detection/train7/weights/best.pt"
        image_path = "/home/faizal/work/gnprc-domi-app/Model/guage_image.jpg"
        
        # Initialize reader with pre-trained model
        print("Loading model...")
        reader = GaugeReader(model_path)

        print("\nProcessing image...")
        print(f"Image path: {image_path}")
        results = reader.predict(image_path)

        if results is None:
            print("\nNo gauge detected with high confidence!")
            print("Check debug_original.jpg and debug_detections.jpg for visualization")
        else:
            print("\nDetection successful!")
            print(f"Gauge Reading: {results['reading']:.1f}")
            print(f"Angle: {results['angle']:.2f} degrees")
            
            output_image = reader.visualize_results(results)
            cv2.imwrite("final_result.jpg", output_image)
            print("Results saved to final_result.jpg")

    except Exception as e:
        print(f"Error: {str(e)}")
        import traceback
        traceback.print_exc()

Loading model...
Model loaded successfully from: /home/faizal/work/gnprc-domi-app/Model/gauge_detection/train7/weights/best.pt

Processing image...
Image path: /home/faizal/work/gnprc-domi-app/Model/guage_image.jpg
Original image shape: (615, 616, 3)

0: 640x640 2 Centers, 1 Gauge, 1 Needle, 4.6ms
Speed: 6.2ms preprocess, 4.6ms inference, 150.0ms postprocess per image at shape (1, 3, 640, 640)

All detections (including low confidence):
Detection 1: Gauge - Confidence: 0.951
Detection 2: Center - Confidence: 0.865
Detection 3: Needle - Confidence: 0.471
Detection 4: Center - Confidence: 0.320

Detection successful!
Gauge Reading: 72.5
Angle: 240.68 degrees
Results saved to final_result.jpg


In [12]:
from ultralytics import YOLO
import cv2
import math
import os

class GaugeReader:
    def __init__(self, model_path):
        try:
            self.model = YOLO(model_path)
            print(f"Model loaded successfully from: {model_path}")
        except Exception as e:
            raise Exception(f"Failed to load model: {str(e)}")

    def calculate_angle(self, center, needle_tip):
        """Calculate angle between vertical line and needle"""
        dx = needle_tip[0] - center[0]
        dy = needle_tip[1] - center[1]
        angle = math.degrees(math.atan2(dy, dx))
        if angle < 0:
            angle += 360
        return angle

    def get_reading_from_angle(self, angle, min_angle=45, max_angle=315, min_value=0, max_value=100):
        """Convert angle to gauge reading"""
        if angle > max_angle:
            angle -= 360
        
        angle_range = max_angle - min_angle
        value_range = max_value - min_value
        
        reading = ((angle - min_angle) / angle_range) * value_range + min_value
        return round(reading, 1)

    def predict(self, image_path):
        """Predict gauge reading from image"""
        if not os.path.exists(image_path):
            raise FileNotFoundError(f"Image not found at: {image_path}")
            
        # Read image
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Could not read image at {image_path}")
            
        # Get predictions
        results = self.model(image)
        
        # Initialize variables
        center = None
        needle_tip = None
        gauge_box = None
        
        # Process predictions
        for result in results:
            boxes = result.boxes
            for box in boxes:
                cls = int(box.cls)
                conf = float(box.conf)
                
                if conf > 0.5:  # Confidence threshold
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    
                    if cls == 0:  # Center
                        center = (int((x1 + x2) / 2), int((y1 + y2) / 2))
                    elif cls == 1:  # Gauge
                        gauge_box = (x1, y1, x2, y2)
                    elif cls == 2:  # Needle
                        needle_tip = (int((x1 + x2) / 2), int((y1 + y2) / 2))
        
        if center and needle_tip:
            angle = self.calculate_angle(center, needle_tip)
            reading = self.get_reading_from_angle(angle)
            
            return {
                'center': center,
                'needle_tip': needle_tip,
                'gauge_box': gauge_box,
                'angle': angle,
                'reading': reading,
                'image': image
            }
        return None

    def visualize_results(self, results):
        """Visualize detection results"""
        if results is None:
            return None
            
        output = results['image'].copy()
        
        if results['gauge_box']:
            x1, y1, x2, y2 = results['gauge_box']
            cv2.rectangle(output, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        if results['center']:
            cv2.circle(output, results['center'], 5, (0, 0, 255), -1)
            
        if results['needle_tip']:
            cv2.circle(output, results['needle_tip'], 5, (255, 0, 0), -1)
            cv2.line(output, results['center'], results['needle_tip'], (255, 0, 0), 2)
            
        cv2.putText(output, 
                   f"Reading: {results['reading']:.1f}", 
                   (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 
                   1, 
                   (0, 0, 255), 
                   2)
        
        return output

# Run inference
if __name__ == "__main__":
    try:
        # Paths
        model_path = "/home/faizal/work/gnprc-domi-app/Model/gauge_detection/train6/weights/best.pt"
        image_path = "/home/faizal/work/gnprc-domi-app/Model/guage_image.jpg"
        output_path = "gauge_result.jpg"

        # Initialize reader with pre-trained model
        print("Loading model...")
        reader = GaugeReader(model_path)

        # Make prediction
        print("Processing image...")
        results = reader.predict(image_path)

        if results is None:
            print("No gauge detected in the image!")
        else:
            # Visualize and save results
            output_image = reader.visualize_results(results)
            cv2.imwrite(output_path, output_image)
            
            print("\nResults:")
            print(f"Gauge Reading: {results['reading']:.1f}")
            print(f"Angle: {results['angle']:.2f} degrees")
            print(f"Visualization saved to: {output_path}")

    except Exception as e:
        print(f"Error: {str(e)}")
        import traceback
        traceback.print_exc()

Loading model...
Model loaded successfully from: /home/faizal/work/gnprc-domi-app/Model/gauge_detection/train6/weights/best.pt
Processing image...

0: 640x640 2 Centers, 1 Gauge, 1 Needle, 3.9ms
Speed: 2.5ms preprocess, 3.9ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
No gauge detected in the image!


In [10]:
import torch
from ultralytics import YOLO
import tensorflow as tf
import numpy as np
import onnx
import onnx2tf
import os
from pathlib import Path
import shutil

class YOLOConverter:
    def __init__(self, model_path, save_dir='converted_models'):
        """
        Initialize converter with path to trained YOLO model
        model_path: Path to YOLOv8 .pt model file
        save_dir: Directory to save converted models
        """
        self.model_path = model_path
        self.save_dir = Path(save_dir)
        self.save_dir.mkdir(parents=True, exist_ok=True)
        
        # Load YOLO model
        self.yolo_model = YOLO(model_path)
    
    def export_to_onnx(self):
        """Export YOLOv8 model to ONNX format"""
        print("Exporting to ONNX...")
        
        # Export to ONNX using ultralytics built-in export
        onnx_path = self.save_dir / 'best.onnx'
        self.yolo_model.export(format='onnx', 
                             dynamic=True,
                             simplify=True)
        
        # Move the exported model to our directory
        shutil.move('best.onnx', onnx_path)
        return str(onnx_path)
    
    def onnx_to_tf(self, onnx_path):
        """Convert ONNX model to TensorFlow SavedModel format"""
        print("Converting ONNX to TensorFlow...")
        
        tf_path = str(self.save_dir / 'tf_model')
        
        # Convert ONNX to TF using onnx2tf
        onnx2tf.convert(
            input_onnx_file_path=onnx_path,
            output_folder_path=tf_path,
            output_signaturedefs=True,
            copy_onnx_input_output_names_to_tflite=True
        )
        
        return tf_path
    
    def optimize_tf_model(self, tf_model_path):
        """Load and optimize TF model for TFLite conversion"""
        print("Optimizing TensorFlow model...")
        
        # Load the SavedModel
        model = tf.saved_model.load(tf_model_path)
        
        # Get concrete function
        concrete_func = model.signatures['serving_default']
        
        return concrete_func
    
    def convert_to_tflite(self, concrete_func, optimization_level='DEFAULT'):
        """Convert to TFLite with specified optimization"""
        print(f"Converting to TFLite with {optimization_level} optimization...")
        
        converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
        
        # Set optimization config
        if optimization_level == 'DEFAULT':
            converter.optimizations = [tf.lite.Optimize.DEFAULT]
        elif optimization_level == 'LATENCY':
            converter.optimizations = [
                tf.lite.Optimize.DEFAULT,
                tf.lite.Optimize.OPTIMIZE_FOR_LATENCY
            ]
        elif optimization_level == 'SIZE':
            converter.optimizations = [
                tf.lite.Optimize.DEFAULT,
                tf.lite.Optimize.OPTIMIZE_FOR_SIZE
            ]
        
        # Enable quantization
        converter.target_spec.supported_ops = [
            tf.lite.OpsSet.TFLITE_BUILTINS,
            tf.lite.OpsSet.SELECT_TF_OPS
        ]
        
        # Convert model
        tflite_model = converter.convert()
        
        # Save model
        tflite_path = self.save_dir / 'model.tflite'
        tflite_path.write_bytes(tflite_model)
        
        return str(tflite_path)
    
    def create_metadata(self, tflite_path):
        """Add metadata to TFLite model"""
        print("Adding metadata...")
        
        metadata = {
            "name": "GaugeReader",
            "description": "Gauge reading detection model converted from YOLOv8",
            "version": "1.0",
            "author": "Converter",
            "license": "MIT"
        }
        
        # Create metadata writer
        writer = tf.lite.experimental.metadata.writer.MetadataWriter()
        
        # Add metadata
        writer.add_metadata(metadata)
        
        # Write metadata to model
        tflite_with_metadata = writer.populate(tflite_path)
        
        # Save model with metadata
        metadata_path = self.save_dir / 'model_with_metadata.tflite'
        metadata_path.write_bytes(tflite_with_metadata)
        
        return str(metadata_path)
    
    def verify_tflite_model(self, tflite_path):
        """Verify TFLite model by running inference"""
        print("Verifying TFLite model...")
        
        # Load TFLite model
        interpreter = tf.lite.Interpreter(model_path=tflite_path)
        interpreter.allocate_tensors()
        
        # Get input and output details
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()
        
        # Create dummy input
        input_shape = input_details[0]['shape']
        dummy_input = np.random.random(input_shape).astype(np.float32)
        
        # Run inference
        interpreter.set_tensor(input_details[0]['index'], dummy_input)
        interpreter.invoke()
        
        # Get output
        output = interpreter.get_tensor(output_details[0]['index'])
        
        print(f"Model verification successful!")
        print(f"Input shape: {input_shape}")
        print(f"Output shape: {output.shape}")
        
        return True
    
    def convert(self, optimization_level='DEFAULT'):
        """Complete conversion pipeline"""
        try:
            # Export to ONNX
            onnx_path = self.export_to_onnx()
            
            # Convert ONNX to TF
            tf_path = self.onnx_to_tf(onnx_path)
            
            # Optimize TF model
            concrete_func = self.optimize_tf_model(tf_path)
            
            # Convert to TFLite
            tflite_path = self.convert_to_tflite(concrete_func, optimization_level)
            
            # Add metadata
            final_model_path = self.create_metadata(tflite_path)
            
            # Verify model
            self.verify_tflite_model(final_model_path)
            
            print(f"\nConversion complete! Model saved at: {final_model_path}")
            return final_model_path
            
        except Exception as e:
            print(f"Error during conversion: {str(e)}")
            raise

In [11]:
# After training your YOLO model
model_path = 'gauge_detection/train6/weights/best.pt'  # Path to your trained YOLO model

# Initialize converter
converter = YOLOConverter(model_path)

# Convert model with desired optimization
# Options: 'DEFAULT', 'LATENCY', or 'SIZE'
tflite_model_path = converter.convert(optimization_level='LATENCY')

Exporting to ONNX...
Ultralytics 8.3.55 🚀 Python-3.12.3 torch-2.5.1+cu124 CPU (AMD Ryzen 5 5600 6-Core Processor)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'gauge_detection/train6/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (6.0 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.45...
ONNX: export success ✅ 9.4s, saved as 'gauge_detection/train6/weights/best.onnx' (11.6 MB)

Export complete (9.6s)
Results saved to /home/faizal/work/gnprc-domi-app/Model/gauge_detection/train6/weights
Predict:         yolo predict task=detect model=gauge_detection/train6/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=gauge_detection/train6/weights/best.onnx imgsz=640 data=/home/faizal/work/gnprc-domi-app/datasets/Analog_Gauge_Meter.v13-v06-crop.yolov8/updated_data.yaml  
Visualize:       https://netron.app
Error during conversion: [Er

FileNotFoundError: [Errno 2] No such file or directory: 'best.onnx'